In [3]:
import pandas as pd
import os
import numpy as np
import cv2
from tqdm import tqdm
import json

In [4]:
# get the possible labels from bdd
categories = set()

with open('/home/ubuntu/huixiong/counter-training/dataset/bdd100k/ori_labels/det_20/det_train.json', 'r') as j:
    bdd_labels = json.loads(j.read())
for img in bdd_labels:
    if 'labels' in img:
        for label in img['labels']:
            categories.add(label['category'])


print(categories)

{'other person', 'trailer', 'bicycle', 'traffic sign', 'rider', 'train', 'car', 'other vehicle', 'motorcycle', 'pedestrian', 'bus', 'truck', 'traffic light'}


In [5]:
cls_mapping = {}
lbl_mapping = {}

for i, c in enumerate(sorted(categories)):
    cls_mapping.update({c: i})
    lbl_mapping.update({i: c})

print(cls_mapping)

{'bicycle': 0, 'bus': 1, 'car': 2, 'motorcycle': 3, 'other person': 4, 'other vehicle': 5, 'pedestrian': 6, 'rider': 7, 'traffic light': 8, 'traffic sign': 9, 'trailer': 10, 'train': 11, 'truck': 12}


In [8]:
# pbar = tqdm(total=len(bdd_labels))

cwd = '/home/ubuntu/huixiong/counter-training/dataset/bdd100k/yolo_labels/val/'
img_path = '/home/ubuntu/huixiong/counter-training/dataset/bdd100k/images/100k/val/'
pbar = tqdm(total=len(bdd_labels))

for img in bdd_labels:

    image = cv2.imread(img_path + img['name'], cv2.IMREAD_COLOR)
    height, width, _ = image.shape

    label_path = cwd + str(img['name']).strip('.jpg') + '.txt'

    if 'labels' in img:
        for l in img['labels']:
            box = l['box2d']

            if os.path.isfile(label_path):
                with open(label_path, "a+") as f:
                    f.write(f"{cls_mapping[l['category']]} {((box['x1'] + box['x2']) / 2 / width):.6f} {((box['y2'] + box['y1']) / 2 / height):.6f} {((box['x2'] - box['x1']) / width):.6f} {((box['y2'] - box['y1']) / height):.6f} \n")
            else:
                with open(label_path, "w+") as f:
                    f.write(f"{cls_mapping[l['category']]} {((box['x1'] + box['x2']) / 2 / width):.6f} {((box['y2'] + box['y1']) / 2 / height):.6f} {((box['x2'] - box['x1']) / width):.6f} {((box['y2'] - box['y1']) / height):.6f} \n")
    else:
        f = open(label_path, 'w+')
        f.close()
    pbar.update(1)
pbar.close()

100%|██████████| 10000/10000 [01:54<00:00, 87.22it/s]
